### Manipulation of PeaskDB de novo-assisted database search results of _T. weisflogii_ rot samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Multiple injections and fragmentation strategies included
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    Files with peptides and PTMs for PTM+cellular compartment x-analysis
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_323-T2dig-all_PEAKS_77/

/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_323-T2dig-all_PEAKS_77


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

323-T2dig-all_PEAKS_77_DB-search-psm.csv
323-T2dig-all_PEAKS_77_DNO.csv
323-T2dig-all_PEAKS_77_DNO.xml
323-T2dig-all_PEAKS_77_peptide.csv
323-T2dig-all_PEAKS_77_peptides.pep.xml
323-T2dig-all_PEAKS_77_peptide.xlsx
323-T2dig-all_PEAKS_77_protein-peptides.csv
323-T2dig-all_PEAKS_77_proteins.csv
323-T2dig-all_PEAKS_77_proteins.fasta
323-T2dig-all_PEAKS_77_proteins-noX.fasta
323-T2dig-all_PEAKS_77_proteins_trypsin_digested_Mass400to6000.txt
323-T2dig-all_PEAKS_77_proteins.xlsx


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup323 = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_323-T2dig-all_PEAKS_77/323-T2dig-all_PEAKS_77_peptide.csv")

# remove redundant rows
peaksdb323 = pd.DataFrame.drop_duplicates(peaksdbdup323)

print(peaksdb323.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb323.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb323['#Spec']
del peaksdb323['Accession']
del peaksdb323['PTM']
del peaksdb323['AScore']

mean_length = peaksdb323['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup323))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb323))

#look at the dataframe
peaksdb323.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area 323_T2_trypsin', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec 323_T2_trypsin', 'Accession', 'PTM', 'AScore'],
      dtype='object')
10.04297224709042
# redundant peaksdb peptides in combined dataframe 1117
# nonredundant peaksdb peptides in combined dataframe 1117


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,LPQVEGTGGDVQPSQDLVR,62.40,1994.0068,19,2.4,998.0131,72.92,118000000,9,24268,20210117_Weissrot_323_T2_trypsin_DDA_120min_2u...
1,VIGQNEAVDAVSNAIR,56.68,1654.8638,16,2.8,828.4415,88.47,104000000,9,30802,20210117_Weissrot_323_T2_trypsin_DDA_120min_2u...
2,AAIGPGIGQGNAAGQAVEGIAR,51.82,1977.0391,22,1.0,660.0209,82.17,179000,8,18310,20210114_Weissrot_323_T2_trypsin_EThcD_120min_...
3,VTDAEIAEVLAR,49.00,1285.6877,12,1.7,643.8522,94.70,115000000,9,33234,20210117_Weissrot_323_T2_trypsin_DDA_120min_2u...
4,Q(-17.03)VHPDTGISK,47.57,1063.5298,10,1.9,532.7732,37.30,2190000,9,11093,20210117_Weissrot_323_T2_trypsin_DDA_120min_2u...


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb323['A'] = peaksdb323['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb323['C'] = peaksdb323['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb323['D'] = peaksdb323['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb323['E'] = peaksdb323['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb323['F'] = peaksdb323['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb323['G'] = peaksdb323['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb323['H'] = peaksdb323['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb323 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb323['I'] = peaksdb323['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb323['K'] = peaksdb323['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb323['L'] = peaksdb323['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb323['M'] = peaksdb323['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb323['N'] = peaksdb323['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb323['P'] = peaksdb323['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb323['Q'] = peaksdb323['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb323['R'] = peaksdb323['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb323['S'] = peaksdb323['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb323['T'] = peaksdb323['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb323['V'] = peaksdb323['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb323['W'] = peaksdb323['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb323['Y'] = peaksdb323['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb323['c-carb'] = peaksdb323['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb323['m-oxid'] = peaksdb323['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a count function to enumerate the # of oxidized K's in each peptide
peaksdb323['k-oxid'] = peaksdb323['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of oxidized P's in each peptide
peaksdb323['p-oxid'] = peaksdb323['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of oxidized R's in each peptide
peaksdb323['r-oxid'] = peaksdb323['Peptide'].apply(lambda x: x.count('R(+15.99)'))

# use a count function to enumerate the # of oxidized Y's in each peptide
peaksdb323['y-oxid'] = peaksdb323['Peptide'].apply(lambda x: x.count('Y(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb323['n-deam'] = peaksdb323['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of methylated K's in each peptide
peaksdb323['k-meth'] = peaksdb323['Peptide'].apply(lambda x: x.count('K(+14.02)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaksdb323['r-meth'] = peaksdb323['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# use a count function to enumerate the # of pyro glu Q's in each peptide
peaksdb323['q-pyro'] = peaksdb323['Peptide'].apply(lambda x: x.count('Q(-17.03)'))

# use a count function to enumerate the # of acetylation of K's in each peptide
peaksdb323['k-acet'] = peaksdb323['Peptide'].apply(lambda x: x.count('K(+42.01)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb323['stripped_peptide'] = peaksdb323['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb323['stripped_length'] = peaksdb323['stripped_peptide'].apply(len)

peaksdb323['NAAF_num.'] = peaksdb323['Area'] / peaksdb323['stripped_length']

# total the number of modifications in sequence
peaksdb323['ptm-total'] = peaksdb323['c-carb'] + peaksdb323['m-oxid'] + peaksdb323['k-oxid'] + peaksdb323['p-oxid'] \
+ peaksdb323['r-oxid'] + peaksdb323['y-oxid'] + peaksdb323['n-deam'] + peaksdb323['k-meth'] + peaksdb323['r-meth'] \
+ peaksdb323['q-pyro'] + peaksdb323['k-acet']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb323['stripped_IL']= peaksdb323['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb323.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_323_T2_trypsin_combine_PTMopt_DB_FDR1.csv")

# check out the results
peaksdb323.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,n-deam,k-meth,r-meth,q-pyro,k-acet,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,LPQVEGTGGDVQPSQDLVR,62.40,1994.0068,19,2.4,998.0131,72.92,118000000,9,24268,...,0,0,0,0,0,LPQVEGTGGDVQPSQDLVR,19,6.210526e+06,0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR,56.68,1654.8638,16,2.8,828.4415,88.47,104000000,9,30802,...,0,0,0,0,0,VIGQNEAVDAVSNAIR,16,6.500000e+06,0,VLGQNEAVDAVSNALR
2,AAIGPGIGQGNAAGQAVEGIAR,51.82,1977.0391,22,1.0,660.0209,82.17,179000,8,18310,...,0,0,0,0,0,AAIGPGIGQGNAAGQAVEGIAR,22,8.136364e+03,0,AALGPGLGQGNAAGQAVEGLAR
3,VTDAEIAEVLAR,49.00,1285.6877,12,1.7,643.8522,94.70,115000000,9,33234,...,0,0,0,0,0,VTDAEIAEVLAR,12,9.583333e+06,0,VTDAELAEVLAR
4,Q(-17.03)VHPDTGISK,47.57,1063.5298,10,1.9,532.7732,37.30,2190000,9,11093,...,0,0,0,1,0,QVHPDTGISK,10,2.190000e+05,1,QVHPDTGLSK


In [6]:
# keep only stripped peptide I/L and NAAF
db_323 = peaksdb323[['stripped_IL', 'Area', 'NAAF_num.']]

db_323.set_index('stripped_IL')

# write modified dataframe to new txt file
db_323.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/I-L_NAAFs/T2_323_trypsin_DB_peptides.csv")

db_323.head()

,stripped_IL,Area,NAAF_num.
0,LPQVEGTGGDVQPSQDLVR,118000000,6.210526e+06
1,VLGQNEAVDAVSNALR,104000000,6.500000e+06
2,AALGPGLGQGNAAGQAVEGLAR,179000,8.136364e+03
3,VTDAELAEVLAR,115000000,9.583333e+06
4,QVHPDTGLSK,2190000,2.190000e+05


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep323moddup = peaksdb323[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep323dup = peaksdb323[["stripped_peptide"]]

# deduplicate both of these lists
pep323mod = pep323moddup.drop_duplicates()
pep323 = pep323dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 323:", len(pep323moddup))
print("Deduplicated modified PeaksDB peptides in 323:", len(pep323mod))
print("Total stripped PeaksDB peptides in 323:", len(pep323dup))
print("Deduplicated stripped PeaksDB peptides in 323:", len(pep323))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep323.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_323_T2_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_323_T2_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_323_T2_trypsin_combine_PTMopt_DB_FDR1_stripped_peptides.fas

# look at the stripped peptides
pep323.head()

Total modified PeaksDB peptides in 323: 1117
Deduplicated modified PeaksDB peptides in 323: 1117
Total stripped PeaksDB peptides in 323: 1117
Deduplicated stripped PeaksDB peptides in 323: 1095


,stripped_peptide
0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR
2,AAIGPGIGQGNAAGQAVEGIAR
3,VTDAEIAEVLAR
4,QVHPDTGISK


### Export txt file of entire (with modification terms) peptides only

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep323moddup = peaksdb323[["Peptide"]]

# deduplicate both of these lists
pep323mod = pep323moddup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep323mod.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PTM-cellular-compartment/to-combine/T2/TW_323_T2_trypsin_combine_PTMopt_DB_FDR1_mod_peptides.txt", header=False, index=False)

# look at the stripped peptides
pep323mod.head()

,Peptide
0,LPQVEGTGGDVQPSQDLVR
1,VIGQNEAVDAVSNAIR
2,AAIGPGIGQGNAAGQAVEGIAR
3,VTDAEIAEVLAR
4,Q(-17.03)VHPDTGISK


## NAAF correction and exporting files with AA and PTM totals:

In [10]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb323['A'].sum(),
        'C': peaksdb323['C'].sum(),
        'D': peaksdb323['D'].sum(),
        'E': peaksdb323['E'].sum(),
        'F': peaksdb323['F'].sum(),
        'G': peaksdb323['G'].sum(),
        'H': peaksdb323['H'].sum(),
        'I': peaksdb323['I'].sum(),
        'K': peaksdb323['K'].sum(),
        'L': peaksdb323['L'].sum(),
        'M': peaksdb323['M'].sum(),
        'N': peaksdb323['N'].sum(),
        'P': peaksdb323['P'].sum(),
        'Q': peaksdb323['Q'].sum(),
        'R': peaksdb323['R'].sum(),
        'S': peaksdb323['S'].sum(),
        'T': peaksdb323['T'].sum(),
        'V': peaksdb323['V'].sum(),
        'W': peaksdb323['W'].sum(),
        'Y': peaksdb323['Y'].sum(),
        'c-carb': peaksdb323['c-carb'].sum(),
        'm-oxid': peaksdb323['m-oxid'].sum(),
        'k-oxid': peaksdb323['k-oxid'].sum(),
        'p-oxid': peaksdb323['p-oxid'].sum(),
        'r-oxid': peaksdb323['r-oxid'].sum(),
        'y-oxid': peaksdb323['y-oxid'].sum(),
        'n-deam': peaksdb323['n-deam'].sum(),
        'k-meth': peaksdb323['k-meth'].sum(),
        'r-meth': peaksdb323['r-meth'].sum(),
        'q-pyro': peaksdb323['q-pyro'].sum(),
        'k-acet': peaksdb323['k-acet'].sum(),
        'Total area': peaksdb323['Area'].sum(),
        'Total length': peaksdb323['stripped_length'].sum()
       }

totalpeaksdb323 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                              'y-oxid', 'n-deam', 'k-meth', 'r-meth', 'q-pyro', \
                                              'k-acet', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb323['% C w/ carb'] = totalpeaksdb323['c-carb'] / totalpeaksdb323['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb323['% M w/ oxid'] = totalpeaksdb323['m-oxid'] / totalpeaksdb323['M'] 

# calculate percentage of K's that are oxidized
totalpeaksdb323['% K w/ oxid'] = totalpeaksdb323['k-oxid'] / totalpeaksdb323['K'] 

# calculate percentage of P's that are oxidized
totalpeaksdb323['% P w/ oxid'] = totalpeaksdb323['p-oxid'] / totalpeaksdb323['P'] 

# calculate percentage of R's that are oxidized
totalpeaksdb323['% R w/ oxid'] = totalpeaksdb323['p-oxid'] / totalpeaksdb323['R'] 

# calculate percentage of Y's that are oxidized
totalpeaksdb323['% Y w/ oxid'] = totalpeaksdb323['y-oxid'] / totalpeaksdb323['Y'] 

# calculate percentage of N's that are deamidated
totalpeaksdb323['% N w/ deam'] = totalpeaksdb323['n-deam'] / totalpeaksdb323['N'] 

# calculate percentage of K's that are methylated
totalpeaksdb323['% K w/ meth'] = totalpeaksdb323['k-meth'] / totalpeaksdb323['K'] 

# calculate percentage of R's that are methylated
totalpeaksdb323['% R w/ meth'] = totalpeaksdb323['r-meth'] / totalpeaksdb323['R'] 

# calculate percentage of Q's that are pyro glu'd
totalpeaksdb323['% Q w/ pyro'] = totalpeaksdb323['q-pyro'] / totalpeaksdb323['Q'] 

# calculate percentage of K's that are acetylation
totalpeaksdb323['% K w/ acet'] = totalpeaksdb323['k-acet'] / totalpeaksdb323['K'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb323['NAAF denom.'] = totalpeaksdb323['Total area'] / totalpeaksdb323['Total length']

# write modified dataframe to new txt file
totalpeaksdb323.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_323_T2_trypsin_combine_PTMopt_DB_FDR1_totals.csv")

totalpeaksdb323.head()

,A,C,D,E,F,G,H,I,K,L,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF denom.
sample total,928,146,930,1148,325,1178,94,645,640,740,...,0.00625,0.076923,0.131148,0.016393,0.032751,0.00625,0.002342,0.072386,0.0,181615.129389


In [14]:
# use the calculated NAAF factor (in totalpeaksdb323 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 181615.129389

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb323['NAAF factor'] = (peaksdb323['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb323_NAAF = peaksdb323[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', 'y-oxid', 'n-deam', \
                              'k-meth', 'r-meth', 'q-pyro', 'k-acet']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb323_NAAF['A-NAAF20'] = peaksdb323_NAAF['A'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['C-NAAF20'] = peaksdb323_NAAF['C'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['D-NAAF20'] = peaksdb323_NAAF['D'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['E-NAAF20'] = peaksdb323_NAAF['E'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['F-NAAF20'] = peaksdb323_NAAF['F'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['G-NAAF20'] = peaksdb323_NAAF['G'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['H-NAAF20'] = peaksdb323_NAAF['H'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['I-NAAF20'] = peaksdb323_NAAF['I'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['K-NAAF20'] = peaksdb323_NAAF['K'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['L-NAAF20'] = peaksdb323_NAAF['L'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['M-NAAF20'] = peaksdb323_NAAF['M'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['N-NAAF20'] = peaksdb323_NAAF['N'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['P-NAAF20'] = peaksdb323_NAAF['P'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['Q-NAAF20'] = peaksdb323_NAAF['Q'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['R-NAAF20'] = peaksdb323_NAAF['R'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['S-NAAF20'] = peaksdb323_NAAF['S'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['T-NAAF20'] = peaksdb323_NAAF['T'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['V-NAAF20'] = peaksdb323_NAAF['V'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['W-NAAF20'] = peaksdb323_NAAF['W'] * peaksdb323['NAAF factor']
peaksdb323_NAAF['Y-NAAF20'] = peaksdb323_NAAF['Y'] * peaksdb323['NAAF factor']

# multiply the NAAF20 factor by the PTMs normalize its abundance by peak area and peptide length

peaksdb323_NAAF['ccarb-NAAF20'] = peaksdb323_NAAF['c-carb'] * peaksdb323_NAAF['NAAF factor']
peaksdb323_NAAF['moxid-NAAF20'] = peaksdb323_NAAF['m-oxid'] * peaksdb323_NAAF['NAAF factor']
peaksdb323_NAAF['koxid-NAAF20'] = peaksdb323_NAAF['k-oxid'] * peaksdb323_NAAF['NAAF factor']
peaksdb323_NAAF['poxid-NAAF20'] = peaksdb323_NAAF['p-oxid'] * peaksdb323_NAAF['NAAF factor']
peaksdb323_NAAF['roxid-NAAF20'] = peaksdb323_NAAF['r-oxid'] * peaksdb323_NAAF['NAAF factor']
peaksdb323_NAAF['yoxid-NAAF20'] = peaksdb323_NAAF['y-oxid'] * peaksdb323_NAAF['NAAF factor']
peaksdb323_NAAF['ndeam-NAAF20'] = peaksdb323_NAAF['n-deam'] * peaksdb323_NAAF['NAAF factor']
peaksdb323_NAAF['kmeth-NAAF20'] = peaksdb323_NAAF['k-meth'] * peaksdb323_NAAF['NAAF factor']
peaksdb323_NAAF['rmeth-NAAF20'] = peaksdb323_NAAF['r-meth'] * peaksdb323_NAAF['NAAF factor']
peaksdb323_NAAF['qpyro-NAAF20'] = peaksdb323_NAAF['q-pyro'] * peaksdb323_NAAF['NAAF factor']
peaksdb323_NAAF['kacet-NAAF20'] = peaksdb323_NAAF['k-acet'] * peaksdb323_NAAF['NAAF factor']

# write the dataframe to a new csv
peaksdb323_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_323_T2_trypsin_combine_PTMopt_DB_FDR1_NAAF.csv")

peaksdb323_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,moxid-NAAF20,koxid-NAAF20,poxid-NAAF20,roxid-NAAF20,yoxid-NAAF20,ndeam-NAAF20,kmeth-NAAF20,rmeth-NAAF20,qpyro-NAAF20,kacet-NAAF20
0,LPQVEGTGGDVQPSQDLVR,34.196085,0,0,2,1,0,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,VIGQNEAVDAVSNAIR,35.789970,3,0,1,1,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,AAIGPGIGQGNAAGQAVEGIAR,0.044800,6,0,0,1,0,6,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,VTDAEIAEVLAR,52.767263,3,0,1,2,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,QVHPDTGISK,1.205847,0,0,1,0,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.205847,0.0


In [15]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb323 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb323_NAAF['NAAF factor'].sum(),
        'A': peaksdb323_NAAF['A-NAAF20'].sum(),
        'C': peaksdb323_NAAF['C-NAAF20'].sum(),
        'D': peaksdb323_NAAF['D-NAAF20'].sum(),
        'E': peaksdb323_NAAF['E-NAAF20'].sum(),
        'F': peaksdb323_NAAF['F-NAAF20'].sum(),
        'G': peaksdb323_NAAF['G-NAAF20'].sum(),
        'H': peaksdb323_NAAF['H-NAAF20'].sum(),
        'I': peaksdb323_NAAF['I-NAAF20'].sum(),
        'K': peaksdb323_NAAF['K-NAAF20'].sum(),
        'L': peaksdb323_NAAF['L-NAAF20'].sum(),
        'M': peaksdb323_NAAF['M-NAAF20'].sum(),
        'N': peaksdb323_NAAF['N-NAAF20'].sum(),
        'P': peaksdb323_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb323_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb323_NAAF['R-NAAF20'].sum(),
        'S': peaksdb323_NAAF['S-NAAF20'].sum(),
        'T': peaksdb323_NAAF['T-NAAF20'].sum(),
        'V': peaksdb323_NAAF['V-NAAF20'].sum(),
        'W': peaksdb323_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb323_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb323_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb323_NAAF['moxid-NAAF20'].sum(),
        'k-oxid': peaksdb323_NAAF['koxid-NAAF20'].sum(),
        'p-oxid': peaksdb323_NAAF['poxid-NAAF20'].sum(),
        'r-oxid': peaksdb323_NAAF['roxid-NAAF20'].sum(),
        'y-oxid': peaksdb323_NAAF['yoxid-NAAF20'].sum(),
        'n-deam': peaksdb323_NAAF['ndeam-NAAF20'].sum(),
        'k-meth': peaksdb323_NAAF['kmeth-NAAF20'].sum(),
        'r-meth': peaksdb323_NAAF['rmeth-NAAF20'].sum(),
        'q-pyro': peaksdb323_NAAF['qpyro-NAAF20'].sum(),
        'k-acet': peaksdb323_NAAF['kacet-NAAF20'].sum()
       }

totalpeaksdb323_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                                   'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                                   'q-pyro', \
                                                   'k-acet'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb323_NAAF['% C w/ carb.'] = totalpeaksdb323_NAAF['c-carb'] / totalpeaksdb323_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb323_NAAF['% M w/ oxid'] = totalpeaksdb323_NAAF['m-oxid'] / totalpeaksdb323_NAAF['M'] 

# calculate NAAF-corrected percentage of K's that are oxidized
totalpeaksdb323_NAAF['% K w/ oxid'] = totalpeaksdb323_NAAF['k-oxid'] / totalpeaksdb323_NAAF['K'] 

# calculate NAAF-corrected percentage of P's that are oxidized
totalpeaksdb323_NAAF['% P w/ oxid'] = totalpeaksdb323_NAAF['p-oxid'] / totalpeaksdb323_NAAF['P'] 

# calculate NAAF-corrected percentage of R's that are oxidized
totalpeaksdb323_NAAF['% R w/ oxid'] = totalpeaksdb323_NAAF['r-oxid'] / totalpeaksdb323_NAAF['R'] 

# calculate NAAF-corrected percentage of Y's that are oxidized
totalpeaksdb323_NAAF['% Y w/ oxid'] = totalpeaksdb323_NAAF['y-oxid'] / totalpeaksdb323_NAAF['Y'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb323_NAAF['% N w/ deam'] = totalpeaksdb323_NAAF['n-deam'] / totalpeaksdb323_NAAF['N'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaksdb323_NAAF['% K w/ meth'] = totalpeaksdb323_NAAF['k-meth'] / totalpeaksdb323_NAAF['K'] 

# calculate NAAF-corrected percentage of R's that are methylated
totalpeaksdb323_NAAF['% R w/ meth'] = totalpeaksdb323_NAAF['r-meth'] / totalpeaksdb323_NAAF['R'] 

# calculate NAAF-corrected percentage of Q's that are pyro glu'd
totalpeaksdb323_NAAF['% Q w/ pyro'] = totalpeaksdb323_NAAF['q-pyro'] / totalpeaksdb323_NAAF['Q'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaksdb323_NAAF['% K w/ acet'] = totalpeaksdb323_NAAF['k-acet'] / totalpeaksdb323_NAAF['K'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb323_NAAF['NAAF check'] = totalpeaksdb323_NAAF['NAAF'] / 181615.129389

# write modified dataframe to new txt file, same name + totals
totalpeaksdb323_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDB/TW_323_T2_trypsin_combine_PTMopt_DB_FDR1_NAAF_totals.csv")

totalpeaksdb323_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF check
sample total,1131.317421,908.198809,128.505455,787.068231,1229.543781,210.934371,915.071303,87.108981,676.119865,523.550866,...,0.003028,0.054775,0.025303,0.002159,0.003015,0.003397,0.000586,0.098085,0.0,0.006229
